# DATA20001 Deep Learning - Group Project
## Text project

**Due Thursday, December 13, before 23:59.**

The task is to learn to assign the correct labels to news articles.  The corpus contains ~850K articles from Reuters.  The test set is about 10% of the articles. The data is unextracted in XML files.

We're only giving you the code for downloading the data, and how to save the final model. The rest you'll have to do yourselves.

Some comments and hints particular to the project:

- One document may belong to many classes in this problem, i.e., it's a multi-label classification problem. In fact there are documents that don't belong to any class, and you should also be able to handle these correctly. Pay careful attention to how you design the outputs of the network (e.g., what activation to use) and what loss function should be used.
- You may use word-embeddings to get better results. For example, you were already using a smaller version of the GloVE  embeddings in exercise 4. Do note that these embeddings take a lot of memory. 
- In the exercises we used e.g., `torchvision.datasets.MNIST` to handle the loading of the data in suitable batches. Here, you need to handle the dataloading yourself.  The easiest way is probably to create a custom `Dataset`. [See for example here for a tutorial](https://github.com/utkuozbulak/pytorch-custom-dataset-examples).

## Download the data

In [49]:
import os
import torch
from torchvision.datasets.utils import download_url
import zipfile

train_path = 'train/'

dl_file='reuters.zip'
dl_url='https://www.cs.helsinki.fi/u/jgpyykko/'
zip_path = os.path.join(train_path, dl_file)
if not os.path.isfile(zip_path):
    download_url(dl_url + dl_file, root=train_path, filename=dl_file, md5=None)

with zipfile.ZipFile(zip_path) as zip_f:
    zip_f.extractall(train_path)
    #os.unlink(zip_path)

The above command downloads and extracts the data files into the `train` subdirectory.

The files can be found in `train/`, and are named as `19970405.zip`, etc. You will have to manage the content of these zips to get the data. There is a readme which has links to further descriptions on the data.

The class labels, or topics, can be found in the readme file called `train/codes.zip`.  The zip contains a file called "topic_codes.txt".  This file contains the special codes for the topics (about 130 of them), and the explanation - what each code means.  

The XML document files contain the article's headline, the main body text, and the list of topic labels assigned to each article.  You will have to extract the topics of each article from the XML.  For example: 
&lt;code code="C18"&gt; refers to the topic "OWNERSHIP CHANGES" (like a corporate buyout).

You should pre-process the XML to extract the words from the article: the &lt;headline&gt; element and the &lt;text&gt;.  You should not need any other parts of the article.

## Your stuff goes here ...

In [50]:
# example http://www2.hawaii.edu/~takebaya/cent110/xml_parse/xml_parse.html
from bs4 import BeautifulSoup
# unzipped a single xml for test!
infile = open("train/477886newsML.xml","r")
contents = infile.read()
soup = BeautifulSoup(contents,'lxml') # use parser lxml as parser xml returns empty list
titles = soup.find_all('title')
for title in titles:
    print(title.get_text())

EU:  REUTER EC REPORT LONG-TERM DIARY FOR APR 7 - DEC 31, 1997.


In [51]:
# single
headline = soup.find('headline')
print(headline.get_text())

REUTER EC REPORT LONG-TERM DIARY FOR APR 7 - DEC 31, 1997.


In [52]:
text = soup.find('text')
print(text.get_text())


****
HIGHLIGHTS
****
AMSTERDAM - The Netherlands hosts summit of European Union leaders (June 16-17).
MADRID - NATO holds summit to set the course for enlargement (July 8 and 9).
LUXEMBOURG - Luxembourg hosts summit of European Union leaders (December 12-13).
APRIL
BRUSSELS (MODIFIED ITEM) - Conference of Bosnian donor countries originally scheduled for April has been POSTPONED to an unspecified date before June.
MONDAY, APRIL 7
NOORDWIJK, Netherlands (NEW ITEM) - EU foreign ministers hold conclave on the inter-governmental conference (second of two days).
NOORDWIJK, Netherlands (EXPANDED ITEM) - EU-Rio Group meeting involving South American countries and Mexico, Panama, Trinidad, Tobago and Costa Rica (To April 8). Honduras, Guyana and chairmen of the Organisation of American States, the InterAmerican Development Bank, Latin American Parliament, the Institute for European-Latin American Relations, the European Investment Bank also attend. Discussions focus on political developments i

In [53]:
codes = soup.find_all('code')
for code in codes:
    print(code)

<code code="EEC">
<editdetail action="confirmed" attribution="Reuters BIP Coding Group" date="1997-04-01"></editdetail>
</code>
<code code="G15">
<editdetail action="confirmed" attribution="Reuters BIP Coding Group" date="1997-04-01"></editdetail>
</code>
<code code="GCAT">
<editdetail action="confirmed" attribution="Reuters BIP Coding Group" date="1997-04-01"></editdetail>
</code>


Refer to topic_codes.txt inside codes.zip, Which defines that 

G15 EUROPEAN COMMUNITY 

GCAT	GOVERNMENT/SOCIAL

EEC is found in region_codes.txt: EEC	EUROPEAN UNION

In [56]:
# read one of the Class-code files
import pandas as pd
import zipfile

zf = zipfile.ZipFile('train/REUTERS_CORPUS_2/codes.zip', 'r') 
colnames=['Code','Description']
df = pd.read_csv(zf.open('topic_codes.txt'), skiprows=2, error_bad_lines=True, 
                 header=None, names=colnames, sep='\t')

In [57]:
df # (the file has 2 first rows as CODE/DESCRIPTION, the extra line is still at row 0)

,Code,Description
0,1POL,CURRENT NEWS - POLITICS
1,2ECO,CURRENT NEWS - ECONOMICS
2,3SPO,CURRENT NEWS - SPORT
3,4GEN,CURRENT NEWS - GENERAL
4,6INS,CURRENT NEWS - INSURANCE
5,7RSK,CURRENT NEWS - RISK NEWS
6,8YDB,TEMPORARY
7,9BNX,TEMPORARY
8,ADS10,CURRENT NEWS - ADVERTISING
9,BNW14,CURRENT NEWS - BUSINESS NEWS


## Save your model

It might be useful to save your model if you want to continue your work later, or use it for inference later.

In [ ]:
torch.save(model.state_dict(), 'model.pkl')

The model file should now be visible in the "Home" screen of the jupyter notebooks interface.  There you should be able to select it and press "download".

## Download test set

The testset will be made available during the last week before the deadline and can be downloaded in the same way as the training set.

## Predict for test set

You will be asked to return your predictions a separate test set.  These should be returned as a matrix with one row for each test article.  Each row contains a binary prediction for each label, 1 if it's present in the image, and 0 if not. The order of the labels is the order of the label (topic) codes.

An example row could like like this if your system predicts the presense of the second and fourth topic:

    0 1 0 1 0 0 0 0 0 0 0 0 0 0 ...
    
If you have the matrix prepared in `y` you can use the following function to save it to a text file.

In [ ]:
np.savetxt('results.txt', y, fmt='%d')